This notebook uses compiled version of the computeVPB function. To compile it run the command

    python setup.py build_ext --inplace

from the `python/` directory

See https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html for some basic tutorial

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import ripser
import persim


In [82]:
import TDAvec
from TDAvec import DiagToPD, computeVPB, computePL, pmin, pmax

In [63]:
X = np.loadtxt("../R/unitCircle.csv", skiprows=1, delimiter=",")
D = ripser.ripser(X, thresh=2)["dgms"]
D[0][-1, 1] = 2


In [102]:
D[1]

array([[0.44604498, 0.45960215],
       [0.40888548, 1.26921213],
       [0.34190655, 0.34717289],
       [0.33821681, 0.34392914],
       [0.33295313, 0.40093198],
       [0.31678182, 0.32426977],
       [0.28439489, 0.31077522],
       [0.28343308, 0.3182652 ],
       [0.26641014, 0.28822109],
       [0.25644588, 0.25812981],
       [0.20289682, 0.2271997 ],
       [0.18578237, 0.28032514],
       [0.14656922, 0.18919227]])

## computeVPB

In [88]:
PD = DiagToPD(D)
ySeqH0 = np.quantile(PD[0][1], np.arange(0, 1.1, 0.2))
xSeqH1 = np.quantile(PD[1][0], np.arange(0, 1.1, 0.2))
ySeqH1 = np.quantile(PD[1][1], np.arange(0, 1.1, 0.2))

vpb0 = computeVPB(PD, homDim=0, xSeq=[], ySeq=ySeqH0)
print(np.allclose(
    vpb0, 
    np.loadtxt("../R/vpb_0.csv", skiprows=1)
    ))
vpb1 = computeVPB(PD, homDim = 1, xSeq=xSeqH1, ySeq=ySeqH1)
vpb1 = np.transpose(vpb1).reshape( (25,))
print(np.allclose(
    vpb1, 
    np.loadtxt("../R/vpb_1.csv", skiprows=1)
    , atol=1e-7))


True
True


## complutePL

In [103]:
scaleSeq = np.linspace(0, 2, 11)
print(np.allclose( computePL(D, 0, scaleSeq), np.loadtxt("../R/pl_0.csv", skiprows=1)))
print(np.allclose(computePL(D, 1, scaleSeq), np.loadtxt("../R/pl_1.csv", skiprows=1)))

True
True


## computePS

In [104]:
def computePS(D, homDim, scaleSeq, p=1):
    x, y = D[homDim][:,0], D[homDim][:,1]
    pp = (y-x)**p
    w = pp/np.sum(pp)

    phi = []
    for k in range(len(scaleSeq)-1):
        alpha1 = pmax(scaleSeq[k], x)
        alpha2 = pmax(scaleSeq[k], (x+y)/2)
        beta1 = pmin(scaleSeq[k+1], (x+y)/2)
        beta2 = pmin(scaleSeq[k+1], y)
        b1 = pmax(0,beta1-alpha1)*((beta1+alpha1)/2-x)
        b2 = pmax(0,beta2-alpha2)*(y-(beta2+alpha2)/2)
        phi.append( np.sum(w*(b1+b2))/(scaleSeq[k+1]-scaleSeq[k]))
    return phi
ps0 = computePS(D, 0, scaleSeq)
ps0_R = np.loadtxt("../R/ps_0.csv", skiprows=1)
print(0,np.allclose(ps0, ps0_R))
ps1 = computePS(D, 1, scaleSeq)
ps1_R = np.loadtxt("../R/ps_1.csv", skiprows=1)
print(1,np.allclose(ps1, ps1_R))

0 True
1 True
